# Template Pattern

Bei der Template-Methode handelt es sich um ein Verhaltensmuster (Behavioral Design Pattern), das das Grundgerüst der Operation definiert und die Details der Implementierung durch die untergeordnete Klasse überlässt. Ihre Unterklassen können die Methodenimplementierungen je nach Bedarf überschreiben, aber der Aufruf muss auf die gleiche Weise erfolgen, wie sie von einer abstrakten Klasse definiert wurde. Es ist eines der am einfachsten zu verstehenden und zu implementierenden Behavioral Design Pattern. Solche Methoden werden häufig bei der Entwicklung von Frameworks verwendet, da sie es uns ermöglichen, ein einziges Stück Code an verschiedenen Stellen wiederzuverwenden, indem wir bestimmte Änderungen vornehmen. Dies führt auch dazu, dass Code-Duplikationen vermieden werden.

<b>Probleme ohne die Template Methode</b>
Stell dir vor du hast ein Webshop, der Bestellprozess ist Standatisiert, der Kunde platziert eine Bestellung, die Bestellung wird verpackt, an die Logistik übergeben und versendet. Dies ist also ein Fester Prozess, aber die Schritte können unterscheidlich sein. Z.b wenn Digitale Produkte verkauft werden, müssen diese als Email versand werden und gehen nicht zur Logistik, während Physikalische Produkte über die Logistik von verschiedenen Logistik-Partner Transportiert werden können. Wir sehen also das der Prozess für die Bestellungen der Gleiche ist, aber die einzelnen Schritte abhängig von dem Produkttyp und der Situation ist.

Schauen wir uns zunächst dein mögliches Programm einmal genauer an.

In [492]:
from typing import List
import random

class Application:

    def __init__(self, trading_strategy = "average") -> None:
        self.trading_strategy = trading_strategy

    def _get_float_list(self, start, end) -> List[float]:
        random_float_list = []
        for _ in range(0, 10):
            x = round(random.uniform(start, end), 2)
            random_float_list.append(x)
        return random_float_list

    def get_actual_price(self) -> float:
        return round(random.uniform(10000,30000), 2)
    
    def get_price_history(self) -> List[float]:
        return self._get_float_list(10000, 30000)

    def connect(self) -> None:
        print(f"Connecting to Crypto exchange...")

    def get_market_data(self, coin: str) -> List[float]:
        return self.get_price_history()

    def list_average(self, l: List[float]) -> float:
        return sum(l) / len(l)

    def should_buy(self, last_price:float, prices: List[float]) -> bool:
        if self.trading_strategy == "minmax":
            return last_price <= min(prices)
        else:
            return last_price < self.list_average(prices)

    def should_sell(self, last_price:float, prices: List[float]) -> bool:
        if self.trading_strategy == "minmax":
            return last_price >= max(prices)
        else:
            return last_price > self.list_average(prices)

    def check_prices(self, coin: str):
        self.connect()
        prices = self.get_market_data(coin)
        last_price = self.get_actual_price()
        should_buy = self.should_buy(last_price, prices)
        should_sell = self.should_sell(last_price, prices)
        if should_buy:
            print(f"You should buy {coin} for {last_price}€!")
        elif should_sell:
            print(f"You should sell {coin} for {last_price}€!")
        else:
            print(f"No action needed for {coin}.")
            

application = Application("minmax")
application.check_prices("BTC/USD")

Connecting to Crypto exchange...
You should buy BTC/USD for 11132.55€!


#### mit Template method

In [494]:
from typing import List
import random
from abc import ABC, abstractmethod


class TradingBot(ABC):

    def _get_float_list(self, start, end) -> List[float]:
        random_float_list = []
        for _ in range(0, 10):
            x = round(random.uniform(start, end), 2)
            random_float_list.append(x)
        return random_float_list

    def get_actual_price(self) -> float:
        return round(random.uniform(10000,30000), 2)
    
    def get_price_history(self) -> List[float]:
        return self._get_float_list(10000, 30000)
    
    def connect(self):
        print(f"Connecting to Crypto exchange...")

    def get_market_data(self, coin: str) -> List[float]:
        return self.get_price_history()

    @abstractmethod
    def should_buy(self, last_price:float, prices: List[float]) -> bool:
        pass

    @abstractmethod
    def should_sell(self, last_price:float, prices: List[float]) -> bool:
        pass

    def check_prices(self, coin: str):
        self.connect()
        prices = self.get_market_data(coin)
        last_price = self.get_actual_price()
        should_buy = self.should_buy(last_price, prices)
        should_sell = self.should_sell(last_price, prices)
        if should_buy:
            print(f"You should buy {coin} for {last_price}€!")
        elif should_sell:
            print(f"You should sell {coin} for {last_price}€!")
        else:
            print(f"No action needed for {coin}.")
            

class AvarageTrader(TradingBot):

    def list_average(self, l: List[float]) -> float:
        return sum(l) / len(l)
    
    def should_buy(self, last_price:float, prices: List[float]) -> bool:
        return last_price < self.list_average(prices)

    def should_sell(self, last_price:float, prices: List[float]) -> bool:
        return last_price > self.list_average(prices)


class MinMaxTrader(TradingBot):

    def should_buy(self, last_price:float, prices: List[float]) -> bool:
        return last_price <= min(prices)

    def should_sell(self, last_price:float, prices: List[float]) -> bool:
        return last_price >= max(prices)


trader = AvarageTrader()
trader.check_prices("BTC/UTC")

Connecting to Crypto exchange...
You should sell BTC/UTC for 27489.95€!
